# Setup

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os
from datetime import datetime

# Import specific packages
import re
from collections import Counter
from scipy.sparse import csr_matrix
from itertools import compress
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Load data

In [2]:
item_cat = pd.read_csv('./data/item_categories.csv')
items = pd.read_csv('./data/items.csv')
sales_train = pd.read_csv('./data/sales_train.csv')
shops = pd.read_csv('./data/shops.csv')
test = pd.read_csv('./data/test.csv')

In [3]:
item_cat.head()

item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4

In [4]:
items.head()

item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40

In [5]:
sales_train.head()

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0

In [6]:
shops.head()

shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран        0
1   !Якутск ТЦ "Центральный" фран        1
2                Адыгея ТЦ "Мега"        2
3  Балашиха ТРК "Октябрь-Киномир"        3
4        Волжский ТЦ "Волга Молл"        4

In [7]:
test.head()

ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268

# Data wrangling

**Merge and flatten dataframe**

First of all, let's join the `items` and `item_cat` together.

In [8]:
items = pd.merge(items, item_cat, how='left', on=['item_category_id'])

**Apply tfidf transform on the `item_name`, `item_category_name` and `shop_name`**

Let convert a collection of raw documents to a matrix of TF-IDF features using the class `sklearn.feature_extraction.text.TfidfVectorizer`, which is equivalent to `CountVectorizer` followed by `TfidfTransformer`.

In [9]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
item_name = pd.DataFrame(tfidf.fit_transform(items['item_name']).toarray())

In [10]:
item_name.head()

0    1    2    3    4    5    6         7    8    9     ...      15  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    ...     0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.637898  0.0  0.0    ...     0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    ...     0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    ...     0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0    ...     0.0   

         16   17   18   19   20   21   22   23        24  
0  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  
1  0.403761  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.483839  
2  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  
3  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  
4  0.000000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  

[5 rows x 25 columns]

Let's create a function to merge the items and item_name together.

In [11]:
def merge_dataframe(df_left, df_right, column_name_prefix):
    for column in df_right.columns.values:
        col = column_name_prefix + str(column)
        df_left[col] = df_right[column]

In [12]:
merge_dataframe(items, item_name, 'item_name')

In [13]:
items.head()

item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  item_name0  \
0                40                            Кино - DVD         0.0   
1                76  Программы - Для дома и офиса (Цифра)         0.0   
2                40                            Кино - DVD         0.0   
3                40                            Кино - DVD         0.0   
4                40                            Кино - DVD         0.0   

   item_name1  item_name2  item_name3  item_name4  item_name5     ...       \
0         0.0         0.0         0.0         0.0         0.0     ...        
1         0.0         0.0         0.0         0.0         0.0     ...        
2         0.0         0.0         0.0         0.0         0.0     ...        
3         0.0         0.0         0.0         0.0         0.0     ...        
4         0.0         0.0         0.0         0.0         0.0     ...        

   item_name15  item_name16  item_name17  item_name18  item_name19  \
0          0.0     0.000000          0.0          0.0          0.0   
1          0.0     0.403761          0.0          0.0          0.0   
2          0.0     0.000000          0.0          0.0          0.0   
3          0.0     0.000000          0.0          0.0          0.0   
4          0.0     0.000000          0.0          0.0          0.0   

   item_name20  item_name21  item_name22  item_name23  item_name24  
0          0.0          0.0          0.0          0.0     0.000000  
1          0.0          0.0          0.0          0.0     0.483839  
2          0.0          0.0          0.0          0.0     0.000000  
3          0.0          0.0          0.0          0.0     0.000000  
4          0.0          0.0          0.0          0.0     0.000000  

[5 rows x 29 columns]

Let's do the same transformation to the item_category_name and shop_name

In [14]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
item_cat_name = pd.DataFrame(tfidf.fit_transform(items['item_category_name']).toarray())

In [15]:
merge_dataframe(items, item_cat_name, 'item_cat_name')

In [16]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
shop_name = pd.DataFrame(tfidf.fit_transform(shops['shop_name']).toarray())

In [17]:
merge_dataframe(shops, shop_name, 'shop_name')

As we notice that there are some negative item_price and negative item_cnt_day. We will filter out those records.

In [18]:
sales_train = sales_train[(sales_train['item_price']>0) & (sales_train['item_cnt_day']>0)]

In [19]:
print('num_records in sales_train:', len(sales_train))

num_records in sales_train: 2928492


Let's check if the (date, shop_id, item_id) are unique.

In [20]:
a = sales_train.groupby(['date', 'shop_id', 'item_id'], as_index=False)['item_cnt_day'].count().rename(columns={'item_cnt_day': 'count'}).sort_values(by=['count'], ascending=False)
a.head()

date  shop_id  item_id  count
2674113  29.01.2013       25    12133      2
669764   07.10.2015       12    21619      2
2778335  30.03.2014       31    16875      2
1073110  12.02.2015       42    21619      2
1976537  21.11.2014       31    16587      2

In [21]:
sales_train[(sales_train['date']=='29.01.2013') & (sales_train['shop_id']==25) & (sales_train['item_id']==12133)]

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
6959  29.01.2013               0       25    12133       889.0           1.0
6960  29.01.2013               0       25    12133      1389.0           1.0

Now that we know the same item could be sold at different price at the same day. Therefore there are two records. As in the test dataset, there is no item_price. Hence let's assume that the item_price of the month to be predicted to be the latest item_price.

**Generate table item_price_latest with the last item_price**

In [22]:
item_price_latest = sales_train.sort_values(by=['date'], ascending=False).groupby(['item_id', 'shop_id'], as_index=False)['item_price'].first()#.rename(columns={'item_price': 'item_price'})
item_price_latest.head()

item_id  shop_id  item_price
0        0       54        58.0
1        1       55      4490.0
2        2       54        58.0
3        3       54        58.0
4        4       54        58.0

**Extract date columns to year, month, day ...**

Now let's extract the `date` column to: `year`, `month`, `day`, `day_of_year`, `weekday`.

In [23]:
sales_train['date'] = sales_train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
sales_train['year'] = sales_train['date'].apply(lambda x: x.year)
sales_train['month'] = sales_train['date'].apply(lambda x: x.month)

**Aggregate on month level**

In [24]:
sales_train1 = sales_train.groupby(['shop_id', 'date_block_num', 'item_id', 'year', 'month'], as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day':'item_cnt_month'})

In [25]:
sales_train1.describe()

shop_id  date_block_num       item_id          year         month  \
count  1.608226e+06    1.608226e+06  1.608226e+06  1.608226e+06  1.608226e+06   
mean   3.280798e+01    1.466502e+01  1.068148e+04  2.013793e+03  6.154981e+00   
std    1.653645e+01    9.542384e+00  6.238856e+03  7.773325e-01  3.455207e+00   
min    0.000000e+00    0.000000e+00  0.000000e+00  2.013000e+03  1.000000e+00   
25%    2.100000e+01    6.000000e+00  5.046000e+03  2.013000e+03  3.000000e+00   
50%    3.100000e+01    1.400000e+01  1.049700e+04  2.014000e+03  6.000000e+00   
75%    4.700000e+01    2.300000e+01  1.606000e+04  2.014000e+03  9.000000e+00   
max    5.900000e+01    3.300000e+01  2.216900e+04  2.015000e+03  1.200000e+01   

       item_cnt_month  
count    1.608226e+06  
mean     2.273154e+00  
std      8.653236e+00  
min      1.000000e+00  
25%      1.000000e+00  
50%      1.000000e+00  
75%      2.000000e+00  
max      2.253000e+03

The model can be influenced by the outliers. Let's use apply clip to the outliers here, where `item_cnt_month` > (mean + 2std) = 19.6. 

In [26]:
sales_train1['item_cnt_month'] = sales_train1['item_cnt_month'].clip(0, 19.6)

**Generate table item_price_avg for each shop and month**

As we need to predict the sales on month level, let's also aggregate the item_price on month level.

In [27]:
item_price_avg = sales_train.groupby(['item_id', 'shop_id', 'year', 'month'], as_index=False)['item_price'].mean()
item_price_avg.head()

item_id  shop_id  year  month  item_price
0        0       54  2014      9        58.0
1        1       55  2014      4      4490.0
2        1       55  2014      7      4490.0
3        1       55  2014      8      4490.0
4        1       55  2014      9      4490.0

**Create training data**

In [28]:
sales_train2 = pd.merge(sales_train1, item_price_avg, how='left', on=['shop_id','item_id','year','month'])
sales_train3 = pd.merge(sales_train2, items, how='left', on=['item_id'])
sales_train4 = pd.merge(sales_train3, shops, how='left', on=['shop_id'])
train = sales_train4

**Create test data**

In [30]:
test2 = pd.merge(test, item_price_latest, how='left', on=['shop_id','item_id'])
test3 = pd.merge(test2, items, how='left', on=['item_id'])
test4 = pd.merge(test3, shops, how='left', on=['shop_id'])
df_test = test4

In [31]:
df_test.head()

ID  shop_id  item_id  item_price  \
0   0        5     5037       749.0   
1   1        5     5320         NaN   
2   2        5     5233       599.0   
3   3        5     5232       599.0   
4   4        5     5268         NaN   

                                           item_name  item_category_id  \
0                     NHL 15 [PS3, русские субтитры]                19   
1                    ONE DIRECTION  Made In The A.M.                55   
2  Need for Speed Rivals (Essentials) [PS3, русск...                19   
3  Need for Speed Rivals (Classics) [Xbox 360, ру...                23   
4               Need for Speed [PS4, русская версия]                20   

                    item_category_name  item_name0  item_name1  item_name2  \
0                           Игры - PS3         0.0    0.000000         0.0   
1  Музыка - CD локального производства         0.0    0.000000         0.0   
2                           Игры - PS3         0.0    0.000000         0.0   
3                      Игры - XBOX 360         0.0    0.588259         0.0   
4                           Игры - PS4         0.0    0.000000         0.0   

      ...       shop_name15  shop_name16  shop_name17  shop_name18  \
0     ...               0.0          0.0          0.0          0.0   
1     ...               0.0          0.0          0.0          0.0   
2     ...               0.0          0.0          0.0          0.0   
3     ...               0.0          0.0          0.0          0.0   
4     ...               0.0          0.0          0.0          0.0   

   shop_name19  shop_name20  shop_name21  shop_name22  shop_name23  \
0          0.0          1.0          0.0          0.0          0.0   
1          0.0          1.0          0.0          0.0          0.0   
2          0.0          1.0          0.0          0.0          0.0   
3          0.0          1.0          0.0          0.0          0.0   
4          0.0          1.0          0.0          0.0          0.0   

   shop_name24  
0          0.0  
1          0.0  
2          0.0  
3          0.0  
4          0.0  

[5 rows x 83 columns]

let's add the month and year to the df_test as well.

In [32]:
train.sort_values(by=['year','month'], ascending=[False, False]).head(1)

shop_id  date_block_num  item_id  year  month  item_cnt_month  \
23204        2              33       31  2015     10             1.0   

       item_price                        item_name  item_category_id  \
23204       399.0  007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD)                37   

      item_category_name     ...       shop_name15  shop_name16  shop_name17  \
23204     Кино - Blu-Ray     ...               0.0          0.0          0.0   

       shop_name18  shop_name19  shop_name20  shop_name21  shop_name22  \
23204          0.0          0.0          0.0      0.49858          0.0   

       shop_name23  shop_name24  
23204          0.0          0.0  

[1 rows x 86 columns]

In [33]:
df_test['year'] = 2015
df_test['month'] = 11
df_test['date_block_num'] = 34

Now we have two dataframe:

- **train**: this would be used for training the model. In the next section, we will spilt this to a training and testing set.
- **df_test**: this would be the month to be predicted.

# Train models

## Train test split: 'future' months as testing set

In terms of how the model is used in reality: we train the model with the historical data, and use it to predict the future. Therefore, a better way to evaluate the model is to leave aside some future months for as test / validation set. For example, if in reality the model should predict the next month's performance, we can leave the lastest month's data as testing_set

In [34]:
train = shuffle(train, random_state=42)

X = train[[col for col in train.columns.values if col not in ['item_name', 'item_category_name', 'shop_name', 'item_cnt_month']]].fillna(0)

y = train['item_cnt_month'].fillna(0)

list_training = list(X['date_block_num']<33)
list_testing = list(X['date_block_num']==33)

X_train2 = X[X['date_block_num']<33]
y_train2 = y[list(compress(y, list_training))].fillna(0)
X_test2 = X[X['date_block_num']==33]
y_test2 = y[list(compress(y, list_testing))].fillna(0)

/Users/jieying/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:705: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [35]:
rf_reg2 = RandomForestRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=42)
rf_reg2.fit(X_train2, y_train2)

y_train2_pred = rf_reg2.predict(X_train2)
y_test2_pred = rf_reg2.predict(X_test2)

In [36]:
rmse_train2 = np.sqrt(mean_squared_error(y_train2, y_train2_pred))
rmse_test2 = np.sqrt(mean_squared_error(y_test2, y_test2_pred))

print('rmse_train:', rmse_train2)
print('rmse_test:', rmse_test2)

rmse_train: 1.2855510378159942
rmse_test: 1.342649379571936


In [37]:
df_test2 = df_test[[col for col in df_test.columns.values if col not in ['ID', 'item_name', 'item_category_name', 'shop_name']]].fillna(0)
test_pred = rf_reg2.predict(df_test2)

In [38]:
test_pred = np.round(test_pred,1)

In [39]:
test['item_cnt_month'] = test_pred
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 4 columns):
ID                214200 non-null int64
shop_id           214200 non-null int64
item_id           214200 non-null int64
item_cnt_month    214200 non-null float64
dtypes: float64(1), int64(3)
memory usage: 6.5 MB


In [40]:
test.to_csv(path_or_buf='./data/result.csv', columns=['ID', 'item_cnt_month'], index=False)

## Optional: other methods of handling item_name

**Stemming**

Let's try some stemming! For this to work, we need the Natural Language Toolkit ([NLTK](http://www.nltk.org/)). 

In [41]:
try:
    import nltk
    
    stemmer = nltk.PorterStemmer()
    for word in ("Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"):
        print(word, '=>', stemmer.stem(word))
except ImportError:
    print("Error: stemming requires the NLTK module.")
    stemmer = None

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


**Create custom transformer**

- TextToWordCounterTransformer
- WordCounterToVector

Now let's create a `custom transformer` that we will use to convert sentense to word counters.

In [42]:
from sklearn.base import BaseEstimator, TransformerMixin


class TextToWordCounterTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, 
                 lower_case=True, 
                 remove_punctuation=True, 
                 replace_numbers=True, 
                 stemming=True):
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_numbers = replace_numbers
        self.stemming = stemming
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed = []
        for text in X:
            if self.lower_case:
                text = text.lower()
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text)
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.stemming and stemmer is not None:
                text = stemmer.stem(text)
            word_counts = Counter(text.split())
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [43]:
class WordCounterToVector(BaseEstimator, TransformerMixin):
    
    def __init__(self, len_vocabulary=25):
        self.len_vocabulary = len_vocabulary
    def fit(self, X, y=None):
        vocabulary_counter = Counter({})
        for counter in X:
            vocabulary_counter = sum((vocabulary_counter, counter), Counter())
        vocabulary_counter = vocabulary_counter.most_common(self.len_vocabulary)
        self.vocabulary = {word: index for index, (word, count) in enumerate(vocabulary_counter, start=1)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for key, val in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary.get(key,0))
                data.append(val)
        a = csr_matrix((data, (rows, cols)), shape=(len(X),self.len_vocabulary+1))
        return a

Now let's create a proprocess pipeline to convert the `item_name` and `item_category_name` into word counter vector.

In [44]:
preprocess_pipeline_item_name = Pipeline((
    ('text_to_word_counter', TextToWordCounterTransformer()),
    ('word_counter_to_vector', WordCounterToVector())
))

In [45]:
preprocess_pipeline_category_name = Pipeline((
    ('text_to_word_counter', TextToWordCounterTransformer()),
    ('word_counter_to_vector', WordCounterToVector())
))